In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
%%appyter hide_code_exec
{% do SectionField(name='section1', title = '1. Input a Gene Symbol or Ensembl ID', subtitle = '', img = 'lncRNA_appyter_logo.png')%}
{% do SectionField(name='section2', title = '2. Options', subtitle = '', img = 'lncRNA_appyter_logo.png')%}

In [ ]:
%%appyter code_exec
{% set query = StringField(name='gene_symbol', label='Gene Symbol/Ensembl ID', default='HOTAIR', description='',section = 'section1') %}
{% set options_fast_compute = BoolField(name='fast_compute', label='Precompute', default='true', description='Precompute will retrieve precomputed results for a faster run time. Select \'No\' to run whole analysis.', section='section2')%}

%%appyter markdown
# Report about the Long Non-coding RNA (lncRNA) {{query.raw_value}}

Based on lncRNA-gene co-expression, this report provides predictions about the biological functions of {{query.raw_value}}, displays the average expression of {{query.raw_value}} across tissues and cell-lines, and lists predictions of small molecules that may specifically up- or down-regulate the expression of {{query.raw_value}}.

In [ ]:
import pandas as pd 
import numpy as np
import h5py as h5
from IPython.display import display,FileLink, HTML, Markdown
import os
import s3fs
from utils import *
from bokeh.io import output_notebook
from bokeh.io import output_file, show
from bokeh.plotting import figure
output_notebook()

In [ ]:
%%appyter code_exec
query = {{ query }}

In [ ]:
# lncRNA of interest
query = query.upper()

%%appyter markdown
### Import gene-lncRNA co-expression matrix

This lncRNA-gene co-expression matrix was computed with Pearson’s correlation coefficient applied to 10,000 randomly selected bulk RNA-seq samples from Recount3 [1]. NOTE: If an Ensembl ID was entered, it will be converted to its corresponding gene symbol if available. 

In [ ]:
# Import lncRNA-gene co-expression matrix
s3 = s3fs.S3FileSystem(anon=True, client_kwargs=dict(endpoint_url='https://s3.appyters.maayanlab.cloud'))
f = h5.File(s3.open('storage/lncRNA_Appyter/Recount3_lncRNA_pcorr.h5', 'rb'), 'r') 
corr =f["data/correlation"]
col_genes = [x.decode('UTF-8') for x in f["meta/columns/genes"]]
row_genes =  [x.decode('UTF-8') for x in f["meta/rows/genes"]]
row_genes_ensembl = [x.decode('UTF-8') for x in f["meta/rows/ensembl"]]

In [ ]:
# Convert input Ensembl ID to gene symbol
ensembl_2_genes = dict(zip(row_genes_ensembl,row_genes))
if query in row_genes_ensembl:
    query_new = ensembl_2_genes[query]
    if query != query_new:
        print('The Ensembl ID ' + query + ' has been converted to the gene symbol ' + query_new )
        query = query_new
        print('Predicting functions for ' + query)
    else:
        print('Predicting functions for ' + query)
else:
    if query in row_genes:
        print('Predicting functions for ' + query )

%%appyter markdown
### Top correlated genes with {{query.raw_value}}

%%appyter markdown
Using the loaded gene-lncRNA correlation matrix, we report the genes that mostly correlate with {{query.raw_value}}.

In [ ]:
# Find most correalted genes and lncRNAs with the input lncRNA
if not os.path.exists("gene_correlations/"):
        os.makedirs("gene_correlations/", exist_ok=True)

# Get index of lncRNA of interest
idx_query = np.where(np.asarray(row_genes) == query)[0][0]

# Ranks genes based on pearson correlation with the lncRNA of interest
lncRNA_coexp = pd.DataFrame(corr[idx_query,:])
lncRNA_coexp.index = col_genes
lncRNA_coexp.columns = ["Pearson's Correlation Coefficient"]
lncRNA_coexp = lncRNA_coexp.sort_values(by="Pearson's Correlation Coefficient", ascending=False)

# save gene correlations to csv file
lncRNA_coexp.to_csv('gene_correlations/'+ query + '_correlated_genes.csv')

In [ ]:
display(lncRNA_coexp[0:20])
display(Markdown(f"*Table 1. Top 20 genes that mostly correlate with {query} ranked by Pearson’s correlation coefficients.*"))
display(FileLink('gene_correlations/' + query + '_correlated_genes.csv', result_html_prefix=str('Download Table 1: ')))

In [ ]:
%%appyter markdown
### Enrichment analysis applied to the top 200 most correlated genes with {{query.raw_value}}

%%appyter markdown
The top 200 most correlated genes with {{query.raw_value}} were submitted to Enrichr [2-4] for enrichment analysis. NOTE: Only genes with official Entrez gene symbols are submitted to Enrichr. Ensembl IDs that do not map to an official gene symbol were dropped.

In [ ]:
# Submit top 200 gene symbols to Enrichr
# For users running the notebook locally: To adjust the number of genes submitted to Enrichr, change the value of n below
n = 200
top_n_genes = [x for x in list(lncRNA_coexp.index) if not x.startswith('ENSG')]
top_n_genes = top_n_genes[0:n]
enrichr_link = Enrichr_API(top_n_genes,str('Top 200 correlated genes with the lncRNA: ' + query))

In [ ]:
display(HTML("Access the enrichment analysis results for {query} 200 most correlated genes here: <a href='{href}'>{link}</a>".format(href=enrichr_link, link = enrichr_link, query=query)))

%%appyter markdown
### Top correlated lncRNAs with {{query.raw_value}}

%%appyter markdown
Below we list the top lncRNAs, out of all 5,050 lncRNAs within our database, that mostly correlate with {{query.raw_value}} based on their Pearson’s correlation coefficients.

In [ ]:
# Download most correlated lncRNAs
lncRNA_lncRNA_coexp = lncRNA_coexp.loc[row_genes]
lncRNA_lncRNA_coexp = lncRNA_lncRNA_coexp.sort_values(by="Pearson's Correlation Coefficient", ascending=False)

# save gene correlations to csv file
lncRNA_coexp.to_csv('gene_correlations/' + query + '_correlated_lncRNAs.csv')

In [ ]:
display(lncRNA_lncRNA_coexp [0:20])
display(Markdown(f"*Table 2. Top 20 lncRNAs that mostly correlate with {query} ranked by Pearson’s correlation coefficients.*"))
display(FileLink('gene_correlations/' + query + '_correlated_lncRNAs.csv', result_html_prefix=str('Download Table 2: ')))

In [ ]:
%%appyter markdown
### Predicted Biological Functions of {{query.raw_value}}

%%appyter markdown
For each Enrichr library, the mean Pearson’s correlation coefficients are calculated between each gene set and {{query.raw_value}}. Terms with a high mean Pearson’s correlation coefficients are prioritized. These terms are predicted to be associated with {{query.raw_value}}.

In [ ]:
%%appyter code_exec
{% if options_fast_compute.raw_value == False %}
# Make function predictions
prediction_libraries = ['MGI_Mammalian_Phenotype_Level_4_2021','GO_Biological_Process_2021','KEGG_2021_Human','DisGeNET']
prediction_libraries = np.array_split(prediction_libraries, int(np.ceil(len(prediction_libraries)/2))) 
for i_group,group in enumerate(prediction_libraries):
    predictions = []
    library_names = []
    for pred_library in group:
        predictions.append(predict_functions(pred_library,lncRNA_coexp,query))
        library_names.append(pred_library.replace('_',' '))
    plot_results(library_names, predictions)
    
    # Save Predictions 
    if not os.path.exists("predicted_functions/"):
        os.makedirs("predicted_functions/", exist_ok=True)
    for ii,prediction in enumerate(predictions):
        prediction.to_csv("predicted_functions/" + library_names[ii]+'_' + query + '.csv')
        display(FileLink("predicted_functions/" + library_names[ii]+'_' + query + '.csv', result_html_prefix=str('Download predictions: ')))
{% endif %}

In [ ]:
%%appyter code_exec
{% if options_fast_compute.raw_value == True %}
# Import pre-computed lncRNA functions
prediction_libraries = ['MGI_Mammalian_Phenotype_Level_4_2021','GO_Biological_Process_2021','KEGG_2021_Human','DisGeNET']
prediction_libraries = np.array_split(prediction_libraries, int(np.ceil(len(prediction_libraries)/2))) 
for i_group,group in enumerate(prediction_libraries):
    predictions = []
    library_names = []
    for pred_library in group:
        precomputed_avg_coexp = pd.read_csv(s3.open('storage/lncRNA_Appyter/'+ pred_library + '_lncRNA_avg_coexpression.csv','rb'),index_col=0)
        precomputed_avg_coexp = pd.DataFrame(precomputed_avg_coexp.loc[query]).sort_values(by=query,ascending = False).reset_index().rename({'index': 'Term', query: 'Mean Pearson Correlation'}, axis='columns')
        predictions.append(precomputed_avg_coexp)
        library_names.append(pred_library.replace('_',' '))
    plot_results(library_names, predictions)
    if i_group == 0:
        display(Markdown(f"*Figure 1. Predicted MGI Mammalian Phenotypes and GO Biological Processes for the lncRNA {query}. Terms are ranked by the mean Pearson correlation between {query} and the associated gene sets.*" ))
    if i_group == 1:
        display(Markdown(f"*Figure 2. Predicted KEGG pathways and DisGeNET disease terms for the lncRNA {query}. Terms are ranked by the mean Pearson correlation between {query} and the associated gene sets.*" ))

    # Save Predictions 
    if not os.path.exists("predicted_functions/"):
        os.makedirs("predicted_functions/", exist_ok=True)
    for ii,prediction in enumerate(predictions):
        prediction.to_csv("predicted_functions/" + library_names[ii]+'_' + query + '.csv')
        display(FileLink("predicted_functions/" + library_names[ii]+'_' + query + '.csv', result_html_prefix=str('Download predictions: ')))
{% endif %}

In [ ]:
%%appyter markdown
### Tissue and Cell Line Expression Levels of {{query.raw_value}}
#### This part of the report provides the Z-score (Normalized Median Expression) for the lncRNA in various tissues and cell lines.

%%appyter markdown
Samples from Recount3 [1] were automatically labelled by their tissue type or cell line of origin. Tissue and cell line samples were log2 transformed and quantile normalized separately. For {{query.raw_value}}, the median expression was then calculated for each tissue type and cell line. Tissues and cell lines with less than 20 samples were removed. Z-scoring was then applied along the lncRNA axis to compare expression levels across all tissues and cell lines.

In [ ]:
# Import z-score data
tissue_expr_zscore  = pd.read_csv(s3.open('storage/lncRNA_Appyter/lncRNA_zscore_median_expr_by_tissue_filtered.csv', 'rb'),header=0, index_col=0) 
tissue_expr_zscore.index = [x.split(',')[0] for x in tissue_expr_zscore.index]

cell_line_expr_zscore = pd.read_csv(s3.open('storage/lncRNA_Appyter/lncRNA_zscore_median_expr_by_cell_line_filtered.csv', 'rb'),header=0, index_col=0) 
cell_line_expr_zscore.index = [x.split(',')[0] for x in cell_line_expr_zscore.index]
cell_line_expr_zscore.columns = [x.upper().replace(' CELL','') for x in cell_line_expr_zscore.columns]

# Create folder for tissue and cell line specific expression
if not os.path.exists("tissue_and_cell_line_expression/"):
    os.makedirs("tissue_and_cell_line_expression/", exist_ok=True)

In [ ]:
# Rank tissues by z-score(median expression)
tissue_specific_lncRNA = pd.DataFrame(tissue_expr_zscore.loc[query])
tissue_specific_lncRNA = tissue_specific_lncRNA.sort_values(by=query,ascending=False)

In [ ]:
plot_bar(tissue_specific_lncRNA ,'All Tissues Ranked by Z-score (Median Expression)','Tissues','Z-score')
display(Markdown(f"*Figure 3. Z-score (median expression) for the lncRNA {query} in various tissue types.*"))
tissue_specific_lncRNA.to_csv("tissue_and_cell_line_expression/" + query + '_tissue_zscore' + '.csv')
display(FileLink("tissue_and_cell_line_expression/" + query + '_tissue_zscore' + '.csv', result_html_prefix=str('Download table for z-score (median expression)in tissue types: ')))

In [ ]:
# Rank cell lines by z-score(median expression)
cell_line_specific_lncRNA = pd.DataFrame(cell_line_expr_zscore.loc[query])
cell_line_specific_lncRNA = cell_line_specific_lncRNA.sort_values(by=query,ascending=False)

In [ ]:
plot_bar(cell_line_specific_lncRNA[0:30],'Top 30 Cell Lines Ranked by Z-score (Median Expression)','Cell Lines','Z-score')
display(Markdown(f"*Figure 4. Z-score (median expression) for the lncRNA {query} in various cell lines.*"))
cell_line_specific_lncRNA.to_csv("tissue_and_cell_line_expression/" + query + '_cell_line_zscore' + '.csv')
display(FileLink("tissue_and_cell_line_expression/" + query + '_cell_line_zscore' + '.csv', result_html_prefix=str('Download table for z-score (median expression)in cell lines: ')))

In [ ]:
%%appyter markdown
### Visualizing all lncRNAs based on their gene expression similarity across tissues

%%appyter markdown
UMAP visualization [5] was applied to 2,000 randomly selected samples (with tissue type labels) from Recount3[1]. Samples were first log2 transformed and quantile normalized (samples as features) before applying UMAP. Each data point is a lncRNA (n=5,050). Use the drop-down menu to color lncRNAs by z-score in a specific tissue. The black arrow is pointing to the location of {{query.raw_value}}.

In [ ]:
umap_tissue_results = pd.read_csv(s3.open('storage/lncRNA_Appyter/umap_tissues.csv', 'rb'),header=0, index_col=0) 
tissue_expr_median_expr = pd.read_csv(s3.open('storage/lncRNA_Appyter/lncRNA_median_expr_by_tissue_filtered.csv', 'rb'),header=0, index_col=0) 

In [ ]:
values_dict_tz = dict()
unique_tissues = np.unique(tissue_expr_zscore.columns)
for t in unique_tissues:
    values_dict_tz[t] = list(tissue_expr_zscore[t])
    
plot_scatter(umap_df=umap_tissue_results, values_dict=values_dict_tz,option_list=list(unique_tissues) ,sample_names=list(umap_tissue_results.index),caption_text='UMAP was applied to 2,000 randomly selected samples (with tissue type labels) from Recount3. Samples were first log2 transformed and quantile normalized (samples as features) before applying UMAP. Each data point is a lncRNA (n=5,050) and are colored by z-score (median expression) in ', figure_counter=5,category_list_dict=None, category=False,dropdown=True,color_by_title='Z-score',highlight_query=query,first_selection=list(tissue_specific_lncRNA.index)[0])

In [ ]:
%%appyter markdown
### Visualizing all lncRNAs based on their gene expression similarity across cell lines 

%%appyter markdown
UMAP visualization [5] was applied to 2,000 randomly selected samples (with cell line labels) from Recount3 [1]. Samples were first log2 transformed and quantile normalized (samples as features) before applying UMAP. Each data point is a lncRNA (n=5,050). Use the drop-down menu to color lncRNAs by z-score in a specific cell line. The black arrow is pointing to the location of {{query.raw_value}}.

In [ ]:
umap_cell_line_results = pd.read_csv(s3.open('storage/lncRNA_Appyter/umap_cell_lines.csv', 'rb'),header=0, index_col=0) 
cell_line_expr_median_expr = pd.read_csv(s3.open('storage/lncRNA_Appyter/lncRNA_median_expr_by_cell_line_filtered.csv', 'rb'),header=0, index_col=0) 

In [ ]:
values_dict_cz = dict()
unique_cell_lines = np.unique(cell_line_expr_zscore.columns)
for t in unique_cell_lines:
    values_dict_cz[t] = list(cell_line_expr_zscore[t])
    
plot_scatter(umap_df=umap_cell_line_results, values_dict=values_dict_cz,option_list=list(unique_cell_lines) ,sample_names=list(umap_cell_line_results.index),caption_text='UMAP was applied to 2,000 randomly selected samples (with cell line labels) from Recount3. Samples were first log2 transformed and quantile normalized (samples as features) before applying UMAP. Each data point is a lncRNA (n=5,050) and are colored by z-score (median expression) in ', figure_counter=6,category_list_dict=None, category=False,dropdown=True,color_by_title='Z-score',highlight_query=query,first_selection=list(cell_line_specific_lncRNA.index)[0])

In [ ]:
%%appyter markdown
### L1000 Small Molecules Predicted to Modulate {{query.raw_value}}

%%appyter markdown
~1.4 million L1000 chemical perturbation gene expression signatures (Level 5) were downloaded from SigCom LINCS (https://maayanlab.cloud/sigcom-lincs) [6]. For each signature, all 5,050 lncRNAs were ranked by mean Pearson’s correlation coefficients and the top 250 lncRNAs were retained for each signature. These lncRNA-L1000 signature associations are reported here. The signatures are ranked by their mean Pearson’s correlation coefficients to modulate the expression of {{query.raw_value}}. These associations are also separated by direction. If {{query.raw_value}} is highly correlated with the up-regulated genes for a specific small molecule, then this small molecule is predicted to up-regulate {{query.raw_value}}.

In [ ]:
%%appyter markdown
### L1000 Small Molecules Predicted to Up-Regulate {{query.raw_value}}

%%appyter markdown
The prioritized small molecules below are predicted to specifically up-regulate {{query.raw_value}}.

In [ ]:
# Load predicted small molecules to modulate the lncRNA
l1000_prediction_file = pd.read_csv(s3.open('storage/lncRNA_Appyter/l1000_sm_lncRNAs_final.tsv', 'rb'),sep='\t',header=None, index_col=0)
l1000_prediction_file = l1000_prediction_file.dropna(axis=1)
if query in list(l1000_prediction_file.index):
    l1000_prediction_file = list(l1000_prediction_file.loc[query])
    drugs_up = []
    drugs_up_corr = []
    drugs_down = []
    drugs_down_corr=[]
    for sig in l1000_prediction_file:
        sig_id = sig.split(',')[0]
        sig_id_corr = float(sig.split(',')[1])
        if sig_id.split(' ')[1] == 'down':
            drugs_down.append(sig_id)
            drugs_down_corr.append(sig_id_corr)
        else:
            drugs_up.append(sig_id)
            drugs_up_corr.append(sig_id_corr)

    up_results = pd.DataFrame({'L1000 Signature ID':drugs_up,'Drug': [x.split('_')[4] for x in drugs_up],'Up/Down':[x.split(' ')[1] for x in drugs_up],'Dose':[x.split(' ')[0].split('_')[-1] for x in drugs_up],'Cell line':[x.split('_')[1] for x in drugs_up],'Time point':[x.split('_')[2] for x in drugs_up],'Mean Pearson Correlation':drugs_up_corr})
    down_results = pd.DataFrame({'L1000 Signature ID':drugs_down,'Drug': [x.split('_')[4] for x in drugs_down],'Up/Down':[x.split(' ')[1] for x in drugs_down],'Dose':[x.split(' ')[0].split('_')[-1] for x in drugs_down],'Cell line':[x.split('_')[1] for x in drugs_down],'Time point':[x.split('_')[2] for x in drugs_down],'Mean Pearson Correlation':drugs_down_corr})
else:
    up_results = pd.DataFrame()
    down_results = pd.DataFrame()


In [ ]:
if len(up_results) > 0:
    display(up_results[0:20])
    display(Markdown(f"*Table 3. L1000 small molecules predicted to up-regulate the lncRNA {query}. L1000 up signatures are prioritized by their Pearson’s correlation coefficients with {query}.*"))

    # Create folder for tissue and cell line specific expression
    if not os.path.exists("l1000_sm_predictions/"):
        os.makedirs("l1000_sm_predictions/", exist_ok=True)
    up_results.to_csv("l1000_sm_predictions/" + query + '_l1000_sm_predictions_up' + '.csv')
    display(FileLink("l1000_sm_predictions/" + query + '_l1000_sm_predictions_up' + '.csv', result_html_prefix=str('Download the table of ranked L1000 small molecules predicted to up-regulate ' + query + ': ')))
else:
    display(Markdown(f"**There are no small molecules predicted to specifically up-regulate the expression of {query}.**"))

In [ ]:
%%appyter markdown
### L1000 Small Molecules Predicted to Down-Regulate {{query.raw_value}}

%%appyter markdown
The prioritized small molecules below are predicted to specifically down-regulate {{query.raw_value}}.

In [ ]:
if len(down_results) > 0:
    display(down_results[0:20])
    display(Markdown(f"*Table 4. L1000 small molecules predicted to down-regulate the lncRNA {query}. L1000 down signatures are prioritized by their Pearson’s correlation coefficients with {query}.*"))
    # Create folder for tissue and cell line specific expression
    if not os.path.exists("l1000_sm_predictions/"):
        os.makedirs("l1000_sm_predictions/", exist_ok=True)
    down_results.to_csv("l1000_sm_predictions/" + query + '_l1000_sm_predictions_down' + '.csv')
    display(FileLink("l1000_sm_predictions/" + query + '_l1000_sm_predictions_down' + '.csv', result_html_prefix=str('Download the table of ranked L1000 small molecules predicted to down-regulate ' + query + ': ')))
else:
    display(Markdown(f"**There are no small molecules predicted to specifically down-regulate the expression of {query}.**"))

In [ ]:
# close h5 file
f.close()

### References
[1] Wilks C, Zheng SC, Chen FY, Charles R, Solomon B, Ling JP, Imada EL, Zhang D, Joseph L, Leek JT: recount3: summaries and queries for large-scale RNA-seq expression and splicing. bioRxiv 2021:2021.2005.2021.445138.

[2] Xie Z, Bailey A, Kuleshov MV, Clarke DJB, Evangelista JE, Jenkins SL, Lachmann A, Wojciechowicz ML, Kropiwnicki E, Jagodnik KM: Gene Set Knowledge Discovery with Enrichr. Current Protocols 2021, 1(3):e90.

[3] Chen EY, Tan CM, Kou Y, Duan Q, Wang Z, Meirelles GV, Clark NR, Ma’ayan A: Enrichr: interactive and collaborative HTML5 gene list enrichment analysis tool. BMC Bioinformatics 2013, 14(1):128.

[4] Kuleshov MV, Jones MR, Rouillard AD, Fernandez NF, Duan Q, Wang Z, Koplev S, Jenkins SL, Jagodnik KM, Lachmann A: Enrichr: a comprehensive gene set enrichment analysis web server 2016 update. Nucleic Acids Research 2016, 44(W1):W90-W97.

[5] McInnes L, Healy J, Melville J: Umap: Uniform manifold approximation and projection for dimension reduction. arXiv preprint arXiv:180203426 2018.

[6] Evangelista et al. SigCom LINCS: Data and Metadata Search Engine for Gene Expression Signatures. 2021. In preparation.
